In [7]:
# Cell 0

network = "custom"  # set this to one of mainnet, yeouido, euljiro, pagoda, or custom

connections = {
"mainnet": {"iconservice": "https://ctz.solidwallet.io",       "nid": 1},
"yeouido": {"iconservice": "https://bicon.net.solidwallet.io", "nid": 3},
"euljiro": {"iconservice": "https://test-ctz.solidwallet.io",  "nid": 2},
"pagoda":  {"iconservice": "https://zicon.net.solidwallet.io", "nid":80},
"custom":  {"iconservice": "http://18.144.108.38:9000",        "nid": 3}}

env = connections[network]

In [8]:
# Cell 1

from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from repeater import retry
from shutil import make_archive
import pickle as pkl
from datetime import datetime
from time import sleep
import json
import os

ICX = 1000000000000000000 # 10**18
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"
ORACLE = "cx61a36e5d10412e03c907a507d1e8c6c3856d9964"

@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result

In [9]:
# Cell 2

icon_service = IconService(HTTPProvider(env["iconservice"], 3))
NID = env["nid"]

In [10]:
# Cell 3

wallet = KeyWallet.load("../../keystores/keystore_test1.json", "test1_Account")
# Balanced test wallet
with open("../../keystores/balanced_test.pwd", "r") as f:
    key_data = f.read()
btest_wallet = KeyWallet.load("../../keystores/balanced_test.json", key_data)
print(icon_service.get_balance(wallet.get_address())/10**18)
print(icon_service.get_balance(btest_wallet.get_address())/10**18)

784368413.2033145
1980342.4945942


In [11]:
print(wallet.get_address())
print(icon_service.get_balance(wallet.get_address()) / 10**18)

hxe7af5fcfd8dfc67530a01a0e403882687528dfcb
784368413.2033145


In [12]:
print(btest_wallet.get_address())
print(icon_service.get_balance(btest_wallet.get_address()) / 10**18)

hx3f01840a599da07b0f620eeae7aa9c574169a4be
1980342.4945942


In [13]:
user1 = KeyWallet.load("../../keystores/user1.json","HelloWorld@1234")
# btest_wallet = KeyWallet.load("./balanced_test.json","HelloWorld@1234")

print(icon_service.get_balance(user1.get_address())/10**18)
print(user1.get_address())

# test2 = hx7a1824129a8fe803e45a3aae1c0e060399546187
private = "0a354424b20a7e3c55c43808d607bddfac85d033e63d7d093cb9f0a26c4ee022"
user2 = KeyWallet.load(bytes.fromhex(private))
print(icon_service.get_balance(user2.get_address())/10**18)
print(user2.get_address())

0.0
hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db
0.0
hx7a1824129a8fe803e45a3aae1c0e060399546187


In [21]:
# The following addresses are those deployed to the private tbears server.

contracts = {'loans': {'zip': 'core_contracts/loans.zip',
  'SCORE': 'cxbbd36ca8d91aefbe1060c3be62fed4d1b848ca85'},
 'staking': {'zip': 'core_contracts/staking.zip',
  'SCORE': 'cxd8e05c1280bc2c32bf53ff61f3bb2e2ecc7d6df5'},
 'dividends': {'zip': 'core_contracts/dividends.zip',
  'SCORE': 'cx7c617e3fca4ba06b6ad203ce113245ae96a9d91e'},
 'reserve': {'zip': 'core_contracts/reserve.zip',
  'SCORE': 'cx70cd5c86f0182d5ac0bd224562d929cd968d9132'},
 'daofund': {'zip': 'core_contracts/daofund.zip',
  'SCORE': 'cxeb91bc377b0620356787d9c9eb68152eb0c62d8a'},
 'rewards': {'zip': 'core_contracts/rewards.zip',
  'SCORE': 'cxfd7511ece084744154fed19dc34732681ad078e6'},
 'dex': {'zip': 'core_contracts/dex.zip',
  'SCORE': 'cx9a3161c778eee2d5758371d3548c5599f76704ec'},
 'governance': {'zip': 'core_contracts/governance.zip',
  'SCORE': 'cx238cd1a1e3a9702d6c9c6dc130719472164db376'},
 'oracle': {'zip': 'core_contracts/oracle.zip',
  'SCORE': 'cx2780eeb8c800ac9886786baae281c3d23bb832fc'},
 'sicx': {'zip': 'token_contracts/sicx.zip',
  'SCORE': 'cxcff8bf80ab213fa9bbb350636a4d68f5cb4fd9c1'},
 'bnUSD': {'zip': 'token_contracts/bnUSD.zip',
  'SCORE': 'cx4c1beaa71b9377100c810c46059ddf5f3da37602'},
 'baln': {'zip': 'token_contracts/baln.zip',
  'SCORE': 'cx3825a86d52c5baf188ff29aa6a7fc2467e285885'},
 'bwt': {'zip': 'token_contracts/bwt.zip',
  'SCORE': 'cx140b49ea041457ebc4cd5e199f5723916bb50021'}}

In [39]:
contracts = {'loans': {'zip': 'core_contracts/loans.zip',
  'SCORE': 'cx2594f06f668fa84804d94c1e5d4b378bfcf23b42'},
 'staking': {'zip': 'core_contracts/staking.zip',
  'SCORE': 'cx00093ced37e5c42b995c4df95fa0ce32290c9a36'},
 'dividends': {'zip': 'core_contracts/dividends.zip',
  'SCORE': 'cx39779100a6556ab3d161dfc95dab3839ac632c9a'},
 'reserve': {'zip': 'core_contracts/reserve.zip',
  'SCORE': 'cx6dd23ec8b6dc6f6c3dec784e200c097f842b8716'},
 'daofund': {'zip': 'core_contracts/daofund.zip',
  'SCORE': 'cxaa4593a3d37c12fe29bc7a9507b0a30593e66f65'},
 'rewards': {'zip': 'core_contracts/rewards.zip',
  'SCORE': 'cxdd1516deed20d0dca0b7ef8c19d2273c48406e86'},
 'dex': {'zip': 'core_contracts/dex.zip',
  'SCORE': 'cx774ddaf4e885782b56c2ddb1df8d8ce3f9ade09e'},
 'governance': {'zip': 'core_contracts/governance.zip',
  'SCORE': 'cx637d21599c04a4c93b390c7f9efb9e7c3a7fcfbb'},
 'oracle': {'zip': 'core_contracts/oracle.zip',
  'SCORE': 'cx61a36e5d10412e03c907a507d1e8c6c3856d9964'},
 'sicx': {'zip': 'token_contracts/sicx.zip',
  'SCORE': 'cx2e94568da72a0e4caf58d2baa1366476f05955ef'},
 'bnUSD': {'zip': 'token_contracts/bnUSD.zip',
  'SCORE': 'cx04afcd84c96ee03b30f6f88a60b2a1dee76c63fd'},
 'bnXLM': {'zip': 'token_contracts/bnXLM.zip',
  'SCORE': 'cx53098409b75594a73fb99badff2b34050a3090b6'},
 'bnDOGE': {'zip': 'token_contracts/bnDOGE.zip',
  'SCORE': 'cx82c8014c8eed1db391109fee2dd0ba11998cd3d2'},
 'baln': {'zip': 'token_contracts/baln.zip',
  'SCORE': 'cx66e56c4ed8a0e7da280867707dcd1ba9322d8c60'},
 'bwt': {'zip': 'token_contracts/bwt.zip',
  'SCORE': 'cx31bd70011e8ce26cb397ec017a339aabc7e7d049'}}

In [22]:
# Cell 6
# Define deploy and send_tx functions

def compress():
    """
    Compress all SCORE folders in the core_contracts and toekn_contracts folders
    """
    deploy = list(contracts.keys())[:]
    for directory in {"../../core_contracts", "../../token_contracts"}:
        with os.scandir(directory) as it:
            for file in it:
                archive_name = directory + "/" + file.name
                if file.is_dir() and file.name in deploy:
                    make_archive(archive_name, "zip", directory, file.name)
                    contracts[file.name]['zip'] = archive_name + '.zip'

def deploy_SCORE(contract, params, wallet, update) -> str:
    """
    contract is of form {'zip': 'core_contracts/governance.zip', 'SCORE': 'cx1d81f93b3b8d8d2a6455681c46128868782ddd09'}
    params is a dicts
    wallet is a wallet file
    update is boolian
    """
    print(f'{contract["zip"]}')
    if update:
        dest = contract['SCORE']
    else:
        dest = GOVERNANCE_ADDRESS
    zip_file = contract['zip']
    step_limit = 4000100000
    deploy_transaction = DeployTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(dest)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content(zip_file))\
        .params(params)\
        .build()

    signed_transaction = SignedTransaction(deploy_transaction, wallet, step_limit)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    print(f'Status: {res["status"]}')
    if len(res["eventLogs"]) > 0:
        for item in res["eventLogs"]:
            print(f'{item} \n')
    if res['status'] == 0:
        print(f'Failure: {res["failure"]}')
    print('')
    return res.get('scoreAddress', '')

def send_tx(dest, value, method, params, wallet):
    """
    dest is the name of the destination contract.
    """
    print('------------------------------------------------------------------------------------------------------------------')
    print(f'Calling {method}, with parameters {params} on the {dest} contract.')
    print('------------------------------------------------------------------------------------------------------------------')
    transaction = CallTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(contracts[dest]['SCORE'])\
        .value(value)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(method)\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    print(f'************************************************** Status: {res["status"]} **************************************************')
    if len(res["eventLogs"]) > 0:
        for item in res["eventLogs"]:
            print(f'{item} \n')
    if res['status'] == 0:
        print(f'Failure: {res["failure"]}')
    return res

def deploy_all(wallet):
    """
    Compress, Deploy and Configure all SCOREs
    """
    compress()

    deploy = list(contracts.keys())[:]
    deploy.remove('oracle')
    deploy.remove('staking')
    deploy.remove('sicx')
    deploy.remove('governance')

    governance = deploy_SCORE(contracts['governance'], {}, wallet, 0)
    contracts['governance']['SCORE'] = governance
    for score in deploy:
        contracts[score]['SCORE'] = deploy_SCORE(contracts[score], {'_governance': governance}, wallet, 0)
    contracts['staking']['SCORE'] = deploy_SCORE(contracts['staking'], {}, wallet, 0)
    contracts['sicx']['SCORE'] = deploy_SCORE(contracts['sicx'], {'_admin': contracts['staking']['SCORE']}, wallet, 0)

    config = list(contracts.keys())[:]
    config.remove('governance')
    addresses = {contract: contracts[contract]['SCORE'] for contract in config}

    txns = [{'contract': 'staking', 'value': 0, 'method': 'setSicxAddress', 'params': {'_address': contracts['sicx']['SCORE']}},
            {'contract': 'governance', 'value': 0, 'method': 'setAddresses', 'params': {'_addresses': addresses}},
            {'contract': 'governance', 'value': 0, 'method': 'launchBalanced', 'params': {}}]

    for tx in txns:
        res = send_tx(tx["contract"], tx["value"], tx["method"], tx["params"], wallet)
        results[f'{tx["contract"]}|{tx["method"]}|{tx["params"]}'] = res

def get_scores_json(contracts):
    """
    Prints out dictionary of SCORE addresses for use in testing UI.
    """
    scores = {}
    for score in contracts:
        scores[score] = contracts[score]['SCORE']
    return json.dumps(scores)

def call_tx(dest: str, method: str, params: dict = {}):
    """
    dest is the name of the destination contract.
    """
    print('------------------------------------------------------------------------------------------------------------------')
    print(f'Reading {method}, with parameters {params} on the {dest} contract.')
    print('------------------------------------------------------------------------------------------------------------------')
    call = CallBuilder()\
        .from_(wallet.get_address())\
        .to(contracts[dest]['SCORE'])\
        .method(method)\
        .params(params)\
        .build()
    print(icon_service.call(call))
    return icon_service.call(call)


In [23]:
# Cell 7
# Deploy and configure Balanced. Print results if anything goes wrong.

results = {}
deploy_all(btest_wallet)
print('------------------------------------------------------------------------------------------------------------------')
print(contracts)
print(get_scores_json(contracts))

../../core_contracts/governance.zip
Status: 1

../../core_contracts/loans.zip


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
Status: 1

../../core_contracts/dividends.zip
Status: 1

../../core_contracts/reserve.zip


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
Status: 1

../../core_contracts/daofund.zip
Status: 1

../../core_contracts/rewards.zip


...

Retrying in 1 seconds...
Status: 1

../../token_contracts/sicx.zip


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
Status: 1

------------------------------------------------------------------------------------------------------------------
Calling setSicxAddress, with parameters {'_address': 'cx7e03cba7cc8f50f844013f454f8363826d66df4f'} on the staking contract.
------------------------------------------------------------------------------------------------------------------
************************************************** Status: 1 **************************************************
------------------------------------------------------------------------------------------------------------------
Calling setAddresses, with parameters {'_addresses': {'loans': 'cx8a531ceeaa7582cd7b812b205617a3591d73ca02', 'staking': 'cx48d8cc175311ce54d4105e11cd5202f2bdf1d973', 'dividends': 'cx653caa4b5c399af4ecf7a80f07abeb703225263a', 'reserve': 'cx73497b98e32aba75caa0301fbc1136021fb2dffe', 'daofund': 'cxdc238f9cdaf57ea8ef30567096b464bbb06c6303', 'rewards': 'cx7e4e1399313c8229d804ff5477b841

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
************************************************** Status: 1 **************************************************
------------------------------------------------------------------------------------------------------------------
Calling launchBalanced, with parameters {} on the governance contract.
------------------------------------------------------------------------------------------------------------------
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cx8a531ceeaa7582cd7b812b205617a3591d73ca02', 'indexed': ['AssetAdded(Address,str,bool)', 'cx7e03cba7cc8f50f844013f454f8363826d66df4f', 'sICX', '0x1'], 'data': []} 

{'scoreAddress': 'cx8a531ceeaa7582cd7b812b205617a3591d73ca02', 'indexed': ['AssetAdded(Address,str,bool)', 'cxf6befaaef07dcd8b07f7ea96d0cf279ec78a0d1a', 'bnUSD', '0x0'], 'data': []} 

{'scoreAddress': 'cx8a531ceeaa7582cd7b812b205617a3591d73ca02', 'indexed': ['AssetAd

In [45]:
# Cell 8
# Deploy or Update a single SCORE

compress()
update = 1
contract = contracts['governance']
params = {}
# params = {'_governance': contracts['governance']['SCORE']}
deploy_SCORE(contract, params, btest_wallet, update)

../../core_contracts/governance.zip


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 2 seconds...
Status: 1



'cx637d21599c04a4c93b390c7f9efb9e7c3a7fcfbb'

In [19]:
call_tx('baln', 'getStakingEnabled', {})


------------------------------------------------------------------------------------------------------------------
Reading getStakingEnabled, with parameters {} on the baln contract.
------------------------------------------------------------------------------------------------------------------
0x1


'0x1'

In [18]:
send_tx('governance', 0 , 'balancetoggleStakingEnabled', {}, btest_wallet)


------------------------------------------------------------------------------------------------------------------
Calling balancetoggleStakingEnabled, with parameters {} on the governance contract.
------------------------------------------------------------------------------------------------------------------


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
************************************************** Status: 1 **************************************************


{'txHash': '0x113ba2055b3af1bf7f0f905f1a817939924ec10857a2ed72779d2e2f57792260',
 'blockHeight': 86575,
 'blockHash': '0xbf42c9e5d53c30aacf898f5c168b06336de83bfaec4a5651f3f22c2d93865639',
 'txIndex': 2,
 'to': 'cx556b9e312a5317c990ff66bcbab93eeb13c5fff7',
 'stepUsed': 158080,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1065560,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

In [42]:
call_tx('baln', 'balanceOf', {'_owner': 'hx27cdf17bf620df87be738757cd9b7af9af7059d6'})


------------------------------------------------------------------------------------------------------------------
Reading balanceOf, with parameters {'_owner': 'hx27cdf17bf620df87be738757cd9b7af9af7059d6'} on the baln contract.
------------------------------------------------------------------------------------------------------------------
0x21e19e0c9bab2400000


'0x21e19e0c9bab2400000'

In [ ]:

send_tx('baln', 0 , 'stake', {'_value': 0 * ICX}, wallet)


In [20]:
send_tx('loans', 1000*ICX, 'depositAndBorrow', {'_asset': 'bnUSD', '_amount': 50}, wallet)


------------------------------------------------------------------------------------------------------------------
Calling addCollateral, with parameters {'_asset': 'bnUSD', '_amount': 50} on the loans contract.
------------------------------------------------------------------------------------------------------------------
************************************************** Status: 0 **************************************************
Failure: {'code': 32, 'message': "USDICX, BandChain, cx7171e2f5653c1b9c000e24228276b8d24e84f10d, Exception: 'NoneType' object has no attribute '_IconScoreBase__is_func_readonly'"}


{'txHash': '0x876bb950e09dc00e33f965bec2547c3eb4ab13f89baf5c3370fa48aff1f238ca',
 'blockHeight': 86637,
 'blockHash': '0xe69f688358a1d1fb904ef6563a0d3e66229782a732212fe58d6eb8a1f2d68a3b',
 'txIndex': 1,
 'to': 'cx2dae159abb954f27a5e1c6031867f2d9f4ad1f32',
 'stepUsed': 607260,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 607260,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0